In [184]:
import cv2
import numpy as np
import os
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
import glob
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import mahotas
import mahotas.features
import milk
from jug import TaskGenerator
from skimage import data
from skimage.feature import hog

%matplotlib inline
#from helper import *
import helper
from sklearn.pipeline import FeatureUnion, Pipeline

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
ls

 Volume in drive E is Data
 Volume Serial Number is 02AA-5720

 Directory of E:\metis\metis_mcnulty

08/01/2015  07:30 PM    <DIR>          .
08/01/2015  07:30 PM    <DIR>          ..
08/01/2015  07:21 PM    <DIR>          .ipynb_checkpoints
08/01/2015  06:07 PM           737,286 bag_of_words.ipynb
08/01/2015  11:29 AM    <DIR>          data
08/01/2015  09:31 AM         1,276,293 exploration.ipynb
08/01/2015  07:29 PM               709 helper.py
08/01/2015  07:30 PM               798 helper.pyc
08/01/2015  09:31 AM            47,895 image_classifier_kevin.ipynb
08/01/2015  07:30 PM             1,988 model.ipynb
08/01/2015  09:47 AM            22,355 OCR for Street View Text - Research.ipynb
08/01/2015  09:31 AM            14,673 ocr_for_characters.ipynb
08/01/2015  09:31 AM             4,867 pair.ipynb
08/01/2015  09:31 AM           118,786 practice.ipynb
08/01/2015  09:31 AM            37,056 Untitled.ipynb
08/01/2015  09:31 AM             1,086 Untitled1.ipynb
08/01/2015  09:31 AM   

In [19]:
train_folder= 'data/train'
train_label='data/trainLabels.csv'

# Load Data

In [10]:
all_train_df= helper.get_cifar_training_data(train_folder,train_label)
all_train_df.head()

,id,label,filename,full_filename
0,1,frog,1.png,data/train\1.png
1,2,truck,2.png,data/train\2.png
2,3,truck,3.png,data/train\3.png
3,4,deer,4.png,data/train\4.png
4,5,automobile,5.png,data/train\5.png


In [49]:
sample_image=all_train_df['full_filename'].ix[54]
img=cv2.imread(sample_image)
img = cv2.cvtColor(img, cv2.cv.CV_BGR2RGB)

In [91]:
img.shape

(32L, 32L, 3L)

In [117]:
img[:,:,0]

array([[ 45,  47,  48, ...,  50,  47,  46],
       [ 45,  47,  48, ...,  55,  51,  51],
       [ 45,  46,  47, ...,  60,  56,  55],
       ..., 
       [ 50,  50,  51, ...,  97,  79,  76],
       [ 50,  49,  51, ...,  81, 103,  72],
       [ 50,  50,  51, ...,  83,  90,  88]], dtype=uint8)

In [104]:
one_image = np.arange(16).reshape((4, 4))
one_image

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [122]:
image.extract_patches_2d(img[:,:,0], (32/2, 32),random_state=0).shape

(17L, 16L, 32L)

In [116]:
patches=image.extract_patches_2d(img[:,:,0], (32/2, 32/2),max_patches=4,random_state=0)
patches[3]

array([[ 48,  50,  51,  54,  55,  57,  58,  59,  60,  60,  59,  55,  51,
         50,  52,  57],
       [ 48,  49,  50,  53,  54,  55,  58,  60,  62,  63,  61,  56,  50,
         49,  51,  54],
       [ 47,  48,  49,  52,  52,  54,  57,  61,  63,  64,  63,  58,  50,
         48,  50,  53],
       [ 46,  47,  48,  50,  51,  53,  57,  61,  63,  64,  63,  60,  54,
         50,  51,  52],
       [ 46,  46,  47,  48,  50,  53,  57,  61,  62,  63,  64,  61,  57,
         52,  53,  53],
       [ 46,  47,  47,  49,  49,  51,  56,  61,  64,  64,  65,  63,  64,
         55,  52,  51],
       [ 47,  47,  47,  48,  48,  51,  56,  61,  65,  66,  62,  76,  99,
        107,  63,  52],
       [ 47,  48,  48,  48,  47,  51,  56,  60,  64,  66,  76,  61,  41,
        108, 105,  55],
       [ 47,  48,  49,  49,  48,  51,  55,  59,  64,  66,  93, 128, 174,
        221, 193,  64],
       [ 48,  49,  50,  50,  49,  50,  55,  60,  63,  67,  67,  38, 111,
        176, 243, 151],
       [ 49,  49,  50,  50,  5

In [199]:
cch_1= helper.ColorChannelStatistics(sub_regions=1)
cch_2= helper.ColorChannelStatistics(sub_regions=1)
cch_1.transform([img])

,cp=B_patch=0_mean,cp=B_patch=0_std,cp=G_patch=0_mean,cp=G_patch=0_std,cp=R_patch=0_mean,cp=R_patch=0_std
0,55.005859,36.553203,94.604492,25.129719,73.3125,30.418372


In [246]:
# Build estimator from PCA and Univariate selection:
combined_features = FeatureUnion([("cch_1", cch_1),("cch_2", cch_2)])

In [247]:
combined_features.transform([img])

array([[ 55.00585938,  36.55320348,  94.60449219,  25.12971853,
         73.3125    ,  30.41837181,  55.00585938,  36.55320348,
         94.60449219,  25.12971853,  73.3125    ,  30.41837181]])

In [202]:
combined_features.get_feature_names()

['cch_1__cp=B_patch=0_mean',
 'cch_1__cp=B_patch=0_std',
 'cch_1__cp=G_patch=0_mean',
 'cch_1__cp=G_patch=0_std',
 'cch_1__cp=R_patch=0_mean',
 'cch_1__cp=R_patch=0_std',
 'cch_2__cp=B_patch=0_mean',
 'cch_2__cp=B_patch=0_std',
 'cch_2__cp=G_patch=0_mean',
 'cch_2__cp=G_patch=0_std',
 'cch_2__cp=R_patch=0_mean',
 'cch_2__cp=R_patch=0_std']

In [227]:
hog_pipeline= Pipeline([("gt", helper.GrayScaleImageTransform())
                        ,("ht", helper.HogStatistics())])


In [220]:
list(hog_pipeline.transform([img,img]))

[array([  2.90106733e-03,   2.09257541e-04,   3.97038268e-04,
          2.09257541e-04,   9.35828171e-04,   2.99525144e-03,
          3.11668552e-03,   2.68631326e-03,   0.00000000e+00,
          7.61823252e-03,   2.09257541e-04,   8.66802002e-04,
          8.91044232e-04,   4.67914086e-04,   2.08732624e-03,
          2.19966844e-03,   4.58347607e-03,   6.12091967e-03,
          1.51925863e-02,   9.21683859e-04,   7.34455913e-04,
         -8.31182386e-20,   1.87165634e-04,  -1.16365534e-18,
          2.93019172e-03,   4.15329170e-03,   2.33103099e-02,
          1.04459739e-03,   1.77285833e-03,   2.85199334e-03,
          1.34222256e-03,   1.77807353e-03,   6.27772622e-04,
          6.02109824e-04,  -4.15591193e-20,   0.00000000e+00,
          2.62807975e-02,   7.35427311e-03,   1.82017443e-02,
          2.57951521e-03,   1.40733629e-03,   1.68832191e-02,
          1.36457412e-02,   6.27772622e-04,   1.99679981e-02,
          1.82812984e-02,   2.41715429e-02,   3.22140082e-02,
        

In [221]:
list(combined_features.transform([img,img]))

[array([ 55.00585938,  36.55320348,  94.60449219,  25.12971853,
         73.3125    ,  30.41837181,  55.00585938,  36.55320348,
         94.60449219,  25.12971853,  73.3125    ,  30.41837181]),
 array([ 55.00585938,  36.55320348,  94.60449219,  25.12971853,
         73.3125    ,  30.41837181,  55.00585938,  36.55320348,
         94.60449219,  25.12971853,  73.3125    ,  30.41837181])]

In [248]:
a=combined_features.transform([img,img])
b=hog_pipeline.transform([img,img])
#np.concatenate((a, b.T), axis=1)

In [257]:
a[0]

array([ 55.00585938,  36.55320348,  94.60449219,  25.12971853,
        73.3125    ,  30.41837181,  55.00585938,  36.55320348,
        94.60449219,  25.12971853,  73.3125    ,  30.41837181])

In [258]:
b[0]

array([  2.90106733e-03,   2.09257541e-04,   3.97038268e-04,
         2.09257541e-04,   9.35828171e-04,   2.99525144e-03,
         3.11668552e-03,   2.68631326e-03,   0.00000000e+00,
         7.61823252e-03,   2.09257541e-04,   8.66802002e-04,
         8.91044232e-04,   4.67914086e-04,   2.08732624e-03,
         2.19966844e-03,   4.58347607e-03,   6.12091967e-03,
         1.51925863e-02,   9.21683859e-04,   7.34455913e-04,
        -8.31182386e-20,   1.87165634e-04,  -1.16365534e-18,
         2.93019172e-03,   4.15329170e-03,   2.33103099e-02,
         1.04459739e-03,   1.77285833e-03,   2.85199334e-03,
         1.34222256e-03,   1.77807353e-03,   6.27772622e-04,
         6.02109824e-04,  -4.15591193e-20,   0.00000000e+00,
         2.62807975e-02,   7.35427311e-03,   1.82017443e-02,
         2.57951521e-03,   1.40733629e-03,   1.68832191e-02,
         1.36457412e-02,   6.27772622e-04,   1.99679981e-02,
         1.82812984e-02,   2.41715429e-02,   3.22140082e-02,
         4.50472001e-02,

In [260]:
np.hstack((a, b))

array([[  5.50058594e+01,   3.65532035e+01,   9.46044922e+01,
          2.51297185e+01,   7.33125000e+01,   3.04183718e+01,
          5.50058594e+01,   3.65532035e+01,   9.46044922e+01,
          2.51297185e+01,   7.33125000e+01,   3.04183718e+01,
          2.90106733e-03,   2.09257541e-04,   3.97038268e-04,
          2.09257541e-04,   9.35828171e-04,   2.99525144e-03,
          3.11668552e-03,   2.68631326e-03,   0.00000000e+00,
          7.61823252e-03,   2.09257541e-04,   8.66802002e-04,
          8.91044232e-04,   4.67914086e-04,   2.08732624e-03,
          2.19966844e-03,   4.58347607e-03,   6.12091967e-03,
          1.51925863e-02,   9.21683859e-04,   7.34455913e-04,
         -8.31182386e-20,   1.87165634e-04,  -1.16365534e-18,
          2.93019172e-03,   4.15329170e-03,   2.33103099e-02,
          1.04459739e-03,   1.77285833e-03,   2.85199334e-03,
          1.34222256e-03,   1.77807353e-03,   6.27772622e-04,
          6.02109824e-04,  -4.15591193e-20,   0.00000000e+00,
        